In [ ]:
import os
import pyspark

from pyspark.sql import SparkSession

os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars file:///E:/Programs/jupyter/venv/drivers/mysql-connector-j/mysql-connector-j-9.1.0.jar pyspark-shell'
path = "E:/Programs/jupyter/venv/drivers/mysql-connector-j/mysql-connector-j-9.1.0.jar"

spark = SparkSession.builder \
    .appName("PySpark Example - MariaDB Example") \
    .master("local") \
    .config("spark.driver.memory", "10g") \
    .config("spark.jars", path)\
    .config("spark.driver.extraClassPath", path)\
    .config("spark.executor.extraClassPath", path)\
    .getOrCreate()


sql = """ 
    select 
        (select count(*) as count from {1} t1 where t1.phone_number = {0}) as t1_c,
        (select count(*) as count from {2} t2 where t2.phone_number = {0}) as t2_c,
        (select count(*) as count from {3} t3 where t3.phone_number = {0}) as t3_c,
        (select count(*) as count from {4} t4 where t4.phone_number = {0}) as t4_c  
"""

sql = sql.format(
    79000000000,
    'test.table1', 
    'test.table2', 
    'test.table3', 
    'test.table4'
)

database = "test"
user = "user"
password = "password"
server = "localhost"
port = 3306
jdbc_url ="jdbc:mysql://localhost:3306/test"
jdbc_driver = "com.mysql.cj.jdbc.Driver"

df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("driver", jdbc_driver) \
    .option("user", user) \
    .option("password", password) \
    .option("query", sql) \
    .load()

df.show()

In [ ]:
import pyspark

from pyspark.sql.functions import col, udf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder \
    .appName("Data helper") \
    .master("local") \
    .config("spark.driver.memory", "10g") \
    .getOrCreate()

path = "F:/path/your/csv/your_csv.csv"
delimiter = ","

df = spark.read \
    .format("csv") \
    .option("header", True) \
    .option("delimiter", delimiter) \
    .csv('file:///' + path)

@udf(returnType=IntegerType()) 
def count_length(str):
    if str == None:
        return 0
    else:
        return len(str)

def type_determ_length_calc(row) -> list:
    my_list = list()
    for col in row:
        my_list.append(len(str(col)))
    return my_list

def reduce_length(acc, x) -> list:
    my_list = list()
    for i in range(0, len(acc)):
        my_list.append(acc[i] if acc[i] > x[i] else x[i])
    return my_list

def create_schema(column_list) -> StructType:
    my_list = list()
    for col in column_list:
        my_list.append(StructField(col, IntegerType(), True))   
    print(my_list)
    return StructType(my_list)

list_data = df.rdd \
            .map(lambda row: type_determ_length_calc(row)) \
            .reduce(lambda acc, x: reduce_length(acc, x))

data = [list_data]
schema = df.columns

df2 = spark.createDataFrame(data=data, schema=schema)
df2.show()